In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm import tqdm, trange
# from pyleetspeak import LeetSpeaker


In [2]:
from pyleetspeak import LeetSpeaker

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\humay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\humay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# pyleetspeak 0.3.9 requires numpy==1.21.5, but you have numpy 1.26.2 which is incompatible.
# pyleetspeak 0.3.9 requires scikit-learn==1.0.2, but you have scikit-learn 1.3.2 which is incompatible.
# synthcity 0.2.9 requires numpy<1.24,>=1.20, but you have numpy 1.26.2 which is incompatible.
# synthcity 0.2.9 requires pandas<2,>=1.4, but you have pandas 2.0.3 which is incompatible.

In [112]:
text_in = "bitch"
leeter = LeetSpeaker(
    change_prb=0.8, change_frq=0.6, mode="basic", seed=0, verbose=False
)
leet_result = leeter.text2leet(text_in)
print(leet_result)

b1tch


In [15]:
#sample 100k words from commonwords
# get leetwords for all words
common_df = pd.read_csv('../commonWords.csv')
common_df = common_df.drop(columns=['count'])
common_df_sample = common_df.sample(frac=0.25)
common_df_sample

,word
232990,cockade
102118,partnersuche
210175,pottercast
260402,glamorize
232391,drey
...,...
37611,taz
55931,adieu
265626,obsid
75141,porfolio


In [16]:
# get pyleet words
leeter = LeetSpeaker(
    change_prb=0.8, change_frq=0.6, mode="basic", seed=0, verbose=False
)
common_leet_dict = {'word':[], 'pyleetspeak':[]}

for i in trange(len(common_df_sample['word'].values)):
    word = common_df_sample['word'].values[i]
    leetword = leeter.text2leet(word)
    common_leet_dict['word'].append(word)
    common_leet_dict['pyleetspeak'].append(leetword)

common_leet_df = pd.DataFrame(common_leet_dict)
common_leet_df.head()

100%|██████████| 83333/83333 [00:02<00:00, 37334.36it/s]


,word,pyleetspeak
0,cockade,c0ck4d3
1,partnersuche,p@rtn3rsuch3
2,pottercast,p0tterc4st
3,glamorize,gl@m0r1ze
4,drey,dr3y


In [17]:
#get leet words using the miner
#first get leet substitutions for a subset of the dataframe, then test accuracy on the other subset
train, test = train_test_split(common_leet_df, test_size=0.30)
test.info()


<class 'pandas.core.frame.DataFrame'>
Index: 25000 entries, 67731 to 13377
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   word         25000 non-null  object
 1   pyleetspeak  25000 non-null  object
dtypes: object(2)
memory usage: 585.9+ KB


In [18]:
#build frequent pattern/substitions list from training dataset
import LeetMining as lm
from pathlib import Path
import json
leet_fp = {}

if Path('pyleetspeak_leetDict.json').exists(): # if dictionary already exists, load from folder
    with open('pyleetspeak_leetDict.json') as json_file:
        leet_fp = json.load(json_file)
else:
    lm.setupLeetDict()
    trainLeet = {'word':[], 'pyleetspeak':[], 'minedLeet':[], 'candidates':[]}

    for i in trange(5000):  #len(train['word'].values)): # running for a smaller subset , most likely will cover all cases
        trainLeet['word'].append(train['word'].values[i])
        trainLeet['pyleetspeak'].append(train['pyleetspeak'].values[i])

        leetWords = lm.getLeetWordList(train['pyleetspeak'].values[i])
        trainLeet['minedLeet'].append(leetWords)
        candidates = []
        for lword in leetWords:
            possibleMatches = lm.getMatchList(lword)
            candidates.append(possibleMatches)
            possibleSubs = lm.getPossibleSubstitutions(lword, possibleMatches)
            lm.updateLeetDict(possibleSubs)
        trainLeet['candidates'].append(candidates)
    train_result_df = pd.DataFrame(trainLeet)
    train_result_df.head()
    leet_fp = lm.getLeetDict()



In [19]:
#mined frequent patterns
leet_fp

{'A': {'0': 372, '3': 402, '4': 266, '@': 284, '_': 278, '1': 348},
 'B': {'0': 40, '4': 20, '3': 38, '1': 36, '_': 40, '@': 20},
 'C': {'3': 35, '@': 26, '4': 12, '_': 24, '1': 29, '0': 38},
 'D': {'0': 46, '3': 47, '4': 33, '@': 33, '_': 51, '1': 34},
 'E': {'3': 514, '0': 299, '1': 245, '_': 216, '@': 145, '4': 158},
 'F': {'0': 29, '@': 23, '_': 27, '1': 21, '3': 23, '4': 13},
 'G': {'3': 53, '_': 41, '0': 45, '1': 39, '@': 30, '4': 18},
 'H': {'3': 34, '0': 44, '@': 24, '_': 28, '4': 23, '1': 29},
 'I': {'1': 358, '0': 254, '@': 120, '3': 246, '4': 107, '_': 190},
 'J': {'3': 11, '_': 10, '1': 8, '@': 7, '4': 3, '0': 9},
 'K': {'0': 26, '3': 23, '@': 22, '_': 18, '4': 11, '1': 31},
 'L': {'0': 69, '3': 61, '@': 44, '_': 38, '1': 53, '4': 29},
 'M': {'4': 31, '3': 41, '@': 33, '_': 45, '1': 41, '0': 48},
 'N': {'0': 74, '3': 83, '@': 37, '4': 42, '_': 61, '1': 57},
 'O': {'0': 397, '3': 289, '@': 134, '1': 265, '4': 146, '_': 211},
 'P': {'4': 30, '3': 36, '_': 38, '1': 33, '@': 48

In [20]:
#collect training results - leet dict and the result table
if not Path('pyleetspeak_leetDict.json').exists():
    with open("pyleetspeak_leetDict.json", "w") as outfile: 
        json.dump(lm.getLeetDict(), outfile)

In [ ]:
# Use the frequent pattern list to detect and de-leetify the pyleetspeak words. Compare with the original and test accuracy.

# Use the existing leetDict to get substitutions and leet transformations for new reply

test_result = {'word':[], 'pyleetspeak':[], 'minedLeet':[], 'BestMatches':[], 'correct': []}

for i in trange(5000):#  len(test['reply'].values)): # running for a smaller subset , most likely will cover all cases
    test_result['word'].append(test['word'].values[i])
    test_result['pyleetspeak'].append(test['pyleetspeak'].values[i])

    leetWords = lm.getLeetWordList(test['pyleetspeak'].values[i])
    test_result['minedLeet'].append(leetWords)
    bestMatches = []
    corrects = []

    for lword in leetWords:
        bestMatch = lm.getBestMatch(lword, leet_fp)
        bestMatches.append(bestMatch)
        if bestMatch.upper() == test['word'].values[i].upper():
            corrects.append("yes")
        else:
            corrects.append("no")
        
    if len(bestMatches) > 0:
        test_result['BestMatches'].append(bestMatches[0])
        test_result['correct'].append(corrects[0])
    else:
        test_result['BestMatches'].append(None)
        test_result['correct'].append(None)


    # polarity_scores = polarity_scores_roberta(newReply)
    # test2_result['Neg'].append(polarity_scores['roberta_neg'])
    # test2_result['Neu'].append(polarity_scores['roberta_neu'])
    # test2_result['Pos'].append(polarity_scores['roberta_pos'])

test_result_df = pd.DataFrame(test_result)
#test_result_df.head()

In [78]:
null = test_result_df[test_result_df['correct'] !='yes']
null = null[null['correct'] !='no']
null = null[null['word'].str.len() >= 15]
#test_result_df['correct']
test_result_df[test_result_df['correct'] =='yes']
#no - try to match words that are already in the dictionary, if can't match, do direct substitution, mostly words  that resemble(close to) some words but not in dictionary
#none - word lengths>=15


,word,pyleetspeak,minedLeet,BestMatches,correct
1,lrapa,lr@p@,[lr@p@],lrApA,yes
4,originals,0r1g1n4ls,[0r1g1n4ls],OrIgInAls,yes
5,aggravating,@ggr4vat1ng,[@ggr4vat1ng],AggrAvatIng,yes
6,gamessiemens,g@m3ss13mens,[g@m3ss13mens],gAmEssIEmens,yes
7,luedenscheid,luedensche1d,[luedensche1d],luedenscheId,yes
...,...,...,...,...,...
4995,indierock,1nd13rock,[1nd13rock],IndIErock,yes
4996,rpgamer,rpg@m3r,[rpg@m3r],rpgAmEr,yes
4997,anaesthesiology,4n43sth3s10l0gy,[4n43sth3s10l0gy],AnAEsthEsIOlOgy,yes
4998,dietikon,di3tik0n,[di3tik0n],diEtikOn,yes


In [80]:
#test_result_df.to_csv("pyleet_accuracy.csv", index=False)

In [96]:
from spellchecker import SpellChecker
#from LeetMining import getBestMatch
spell = SpellChecker()
spell.candidates('early-stage') # hyphenated words don't have candidates
#lm.getBestMatch('early-stage', leet_fp)

In [132]:
#note think about running test for twitter again ? if I have time, made some changes in the bestmatches function, like defaulting to direct substitution
#might not be a good idea, fp list is not good enough to be relying directly on the list
#todo: save accuracy test df for result analysys later, run the same test for profanity list, run test on reddit, start with the dictionary
#leet_fp2['0']

'A'